In [1]:
import json
import numpy as np
import copy
import transformers
import torch
import tqdm
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import json
import plotly.graph_objects as go
import plotly.offline as pyo

In [86]:
sel_seed=42
data=json.load(open('../data/new_evals_03_03/for_manual_inspection/1.5b/pre_temp=0.1_n=64.json','r'))
solve_probs=np.array([d["corrects_verl_batched_responses"] for d in data]).astype(np.float32).mean(-1)

sel=(solve_probs>0.4) & (solve_probs<0.6)
#sel=(solve_probs==0.0)
sel_inds=np.where(sel)[0]
np.random.seed(sel_seed)
sel_inds=np.random.choice(sel_inds, 10, replace=False)
sel_inds,sel.sum()

(array([437, 244, 413, 254, 175, 179, 455, 415, 220,  10]), 30)

In [120]:
choice=7
#
if choice==0:
    file_path='../data/new_evals_03_03/for_manual_inspection/1.5b/pre_temp=0.1_n=64.json'
    save_path="./trees/1_5b_pre_sp=mid_nq=10.html"
    inds=[437, 244, 413, 254, 175, 179, 455, 415, 220,  10]
    model_name='Qwen/Qwen2.5-1.5B-Instruct'
elif choice==1:
    file_path='../data/new_evals_03_03/for_manual_inspection/1.5b/post_temp=0.1_n=64.json'
    save_path="./trees/1_5b_post_sp=mid_nq=10.html"
    inds=[437, 244, 413, 254, 175, 179, 455, 415, 220,  10]
    model_name='Qwen/Qwen2.5-1.5B-Instruct'
elif choice==2:
    file_path='../data/new_evals_03_03/for_manual_inspection/1.5b/pre_temp=0.1_n=64.json'
    save_path="./trees/1_5b_pre_sp=zero_nq=10.html"
    inds=[454,  11, 369, 156, 104, 431, 101, 138,  60, 401]
    model_name='Qwen/Qwen2.5-1.5B-Instruct'
elif choice==3:
    file_path='../data/new_evals_03_03/for_manual_inspection/1.5b/post_temp=0.1_n=64.json'
    save_path="./trees/1_5b_post_sp=zero_nq=10.html"
    inds=[454,  11, 369, 156, 104, 431, 101, 138,  60, 401]
    model_name='Qwen/Qwen2.5-1.5B-Instruct'
elif choice==4:
    file_path='../data/new_evals_03_03/for_manual_inspection/7b/pre_temp=0.1_n=64.json'
    save_path="./trees/7b_pre_sp=mid_nq=10.html"
    inds=[ 19, 219, 326,  41, 281, 394, 123, 340, 380, 429]
    model_name='Qwen/Qwen2.5-7B-Instruct'
elif choice==5:
    file_path='../data/new_evals_03_03/for_manual_inspection/7b/post_temp=0.1_n=64.json'
    save_path="./trees/7b_post_sp=mid_nq=10.html"
    inds=[ 19, 219, 326,  41, 281, 394, 123, 340, 380, 429]
    model_name='Qwen/Qwen2.5-7B-Instruct'
elif choice==6:
    file_path='../data/new_evals_03_03/for_manual_inspection/7b/pre_temp=0.1_n=64.json'
    save_path="./trees/7b_pre_sp=zero_nq=10.html"
    inds=[227, 383, 425, 166, 401,  96, 214, 467,  80, 306]
    model_name='Qwen/Qwen2.5-7B-Instruct'
elif choice==7:
    file_path='../data/new_evals_03_03/for_manual_inspection/7b/post_temp=0.1_n=64.json'
    save_path="./trees/7b_post_sp=zero_nq=10.html"
    inds=[227, 383, 425, 166, 401,  96, 214, 467,  80, 306]
    model_name='Qwen/Qwen2.5-7B-Instruct'
else:
    raise ValueError("Invalid choice")
print(file_path, save_path, inds, model_name)

../data/new_evals_03_03/for_manual_inspection/7b/post_temp=0.1_n=64.json ./trees/7b_post_sp=zero_nq=10.html [227, 383, 425, 166, 401, 96, 214, 467, 80, 306] Qwen/Qwen2.5-7B-Instruct


## This prepares the data for actual math solution generations

In [121]:
data=json.load(open(file_path,'r'))

In [122]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name,padding_side="right")
print("Selected",len(inds),"out of",len(data))

Selected 10 out of 500


In [123]:
sampling_data=[]
for ind in tqdm.tqdm(inds):
    uid=data[ind]["id"]
    prompt=data[ind]["prompt"]
    gt_answer=data[ind]["answer"]
    model_answers=data[ind]["verl_model_answers"]
    for i in range(len(model_answers)):
        if model_answers[i] is None or model_answers[i]=="None" or model_answers[i]=="":
            model_answers[i]="ERROR"

    description=(
        "UNIQUE ID:\n"+uid+"\n\n\n"
        "PROMPT:\n"+prompt+"\n\n\n"
        "ANSWER:\n"+gt_answer
    )

    responses=data[ind]["responses"]
    token_idss=tokenizer(responses,return_tensors="pt",padding=True).input_ids.tolist()
    scores=data[ind]['corrects_verl_batched_responses']
    
    addtionals=[{
        "model_answer":model_answer,
    }
    for model_answer in model_answers]

    element={
        "description":description,
        "ids":token_idss,
        "scores":scores,
        "additionals":addtionals,
    }
    sampling_data.append(element)

def get_label(ids):
    if len(ids)==0:
        return ""
    string=tokenizer.decode(ids,skip_special_tokens=False)
    string=string.replace("<|endoftext|>","")
    return string

100%|██████████| 10/10 [00:00<00:00, 10.74it/s]


In [124]:
def make_tree(ids):
    ids = np.array(ids)
    n, l = ids.shape
    nodelists = [[] for _ in range(n)]
    node_to_idlist = {}

    groups = [(0, np.arange(n), 0)]
    next_node = 1  # next available node id

    # Process each group until no groups remain.
    while groups:
        new_groups = []
        for node, indices, start in groups:
            # Optimization: if the remainder of the sequence is identical across the group,
            # then add the entire segment at once.
            if start < l and np.all(ids[indices, start:] == ids[indices[0], start:]):
                common_segment = ids[indices[0], start:].astype(int).tolist()
                node_to_idlist[node] = common_segment
                for i in indices:
                    nodelists[i].append(node)
                continue  # This group is complete.
            
            col = start
            # Advance until the sequences in the group differ, or we reach the end.
            while col < l:
                # Check if the column is identical.
                if np.all(ids[indices, col] == ids[indices[0], col]):
                    # Optionally, check if from this column onward all values are identical.
                    if np.all(ids[indices, col:] == ids[indices[0], col:]):
                        col = l
                        break
                    col += 1
                else:
                    break

            # Record the contiguous segment from start to col (noninclusive)
            common_segment = ids[indices[0], start:col].astype(int).tolist()
            node_to_idlist[node] = common_segment
            for i in indices:
                nodelists[i].append(node)

            # If we have not reached the end, then the group splits.
            if col < l:
                groups_dict = {}
                for i in indices:
                    val = ids[i, col]
                    groups_dict.setdefault(val, []).append(i)
                for val, idxs in groups_dict.items():
                    new_groups.append((next_node, np.array(idxs), col))
                    next_node += 1
            # (If col == l, the sequence is complete.)
        groups = new_groups
    return nodelists, node_to_idlist

def check_reconstruction(ids,nodelists,node_to_idlist):
    ids=np.array(ids)
    ids_rec = []
    for nodelist in nodelists:
        ids_rec_ = []
        for node in nodelist:
            ids_rec_.extend(node_to_idlist[node])
        ids_rec.append(ids_rec_)
    ids_rec = np.array(ids_rec)
    assert np.all(ids == ids_rec), "Reconstruction failed"

def make_json_tree(nodelists, node_to_idlist, description, scores, additionals):
    """
    Build a JSON tree representation from the tree data.
    
    Parameters:
      nodelists: list of lists. Each inner list is the sequence of node ids visited by one input sequence.
      node_to_idlist: dict mapping node id to a contiguous list of ints (the node's content).
      description: string. Will be added to the root node.
      scores: list of length n (one per input sequence).
      additionals: list of dict of length n (one per input sequence).
      
    Returns:
      A JSON-serializable dictionary representing the tree.
    """
    # Create the root node (node id 0) with the additional "description" field.
    tree = {
        "node": 0,
        "idlist": node_to_idlist[0],
        "description": description,
        "children": []
    }
    
    # Insert each path from nodelists into the tree.
    for seq_idx, path in enumerate(nodelists):
        current = tree
        # Assume each path starts at the root (node id 0); iterate from the second element.
        for node_id in path[1:]:
            # Try to find an existing child with this node id.
            child = next((child for child in current["children"] if child["node"] == node_id), None)
            if child is None:
                # Create a new child node.
                child = {
                    "node": node_id,
                    "idlist": node_to_idlist[node_id],
                    "children": []
                }
                current["children"].append(child)
            current = child
        # At the end of the path, mark current as a leaf by recording the sequence index.
        current.setdefault("seq_indices", []).append(seq_idx)
    
    # Recursively annotate leaf nodes with "scores" and "additionals" then remove temporary keys.
    def annotate_leaves(node):
        if not node.get("children"):
            if "seq_indices" in node:
                # Gather the scores and additionals for the sequences that ended here.
                node["scores"] = [scores[i] for i in node["seq_indices"]]
                node["additionals"] = [additionals[i] for i in node["seq_indices"]]
            # Remove the temporary seq_indices key.
            node.pop("seq_indices", None)
        else:
            for child in node["children"]:
                annotate_leaves(child)
                
    annotate_leaves(tree)
    
    return tree

def annotate_tree(tree, get_label):
    """
    Recursively annotate the JSON tree with additional fields.
    
    For each node in the tree:
      1. Assign "label" as get_label(node["idlist"]).
      2. Compute "count": total number of sequences below (leaves),
         and "counts": a list with the count for each child.
      3. Compute "total_score" as the sum of scores from all leaves below,
         and "avg_score" = total_score/count (or 0 if count is 0).
      4. Assign "color" based on avg_score (using a red-to-green gradient,
         where avg_score 0.0 maps to red (#ff0000) and 1.0 to green (#00ff00)).
      5. After recursion, perform a DFS pass to assign "text" attribute.
         The text is built by concatenating the parent's chain (in black)
         with an empty blue delimiter, and the current node's label in red.
    
    Parameters:
      tree: dict, a JSON-serializable tree. Leaves are expected to have a "scores" key.
      get_label: function mapping an idlist (list of ints) to a string label (can handle empty lists).
      
    Returns:
      A tuple (count, total_score) for the current node.
    """
    def recursive_annotate(node):
        # 1. Compute label for this node.
        node["label"] = get_label(node.get("idlist", []))
        
        # 2. If the node is a leaf, use its "scores" list.
        if "children" not in node or not node["children"]:
            if "scores" in node:
                count = len(node["scores"])
                total_score = sum(node["scores"])
            else:
                count = 0
                total_score = 0
            node["count"] = count
            node["counts"] = []
        else:
            child_counts = []
            total_score = 0
            for child in node["children"]:
                c, ts = recursive_annotate(child)
                child_counts.append(c)
                total_score += ts
            count = sum(child_counts)
            node["count"] = count
            node["counts"] = child_counts
        
        node["total_score"] = total_score
        node["avg_score"] = total_score / count if count > 0 else 0
        
        # 4. Assign a color based on avg_score: linear interpolation from red (0.0) to green (1.0)
        s = node["avg_score"]
        r = int(255 * (1 - s))
        g = int(255 * s)
        b = 0
        node["color"] = f"#{r:02x}{g:02x}{b:02x}"
        
        return count, total_score

    # Run the recursive annotation.
    recursive_annotate(tree)
    
    # 5. DFS pass to assign "text" attribute for each node.
    # Each node's text is parent's black chain + blue delimiter + current label in red.
    delimiter = "<span style='color:blue;'></span>"
    stack = [(tree, "")]
    while stack:
        node, header = stack.pop()
        # Create the text attribute: parent's header + delimiter + current label in red.
        node["text"] = header + delimiter + f"<span style='color:red;'>{node['label']}</span>"
        # For children, pass along parent's header updated to include this node's label in black.
        new_header = header + delimiter + f"<span style='color:black;'>{node['label']}</span>"
        for child in node.get("children", []):
            stack.append((child, new_header))
    
    return tree["count"], tree["total_score"]

def get_tree(data_element, get_label):
    description = data_element["description"]
    ids = data_element["ids"]
    scores = data_element["scores"]
    additionals = data_element["additionals"]

    nodelists, node_to_idlist = make_tree(ids)
    check_reconstruction(ids, nodelists, node_to_idlist)

    tree= make_json_tree(nodelists, node_to_idlist, description, scores, additionals)
    annotate_tree(tree, get_label)
    return tree

In [125]:
trees=[]
for data_element in tqdm.tqdm(sampling_data):
    tree = get_tree(data_element, get_label)
    trees.append(tree)

100%|██████████| 10/10 [00:00<00:00, 10.52it/s]


In [126]:
def compress_tree(tree):
    """
    Recursively compress an annotated JSON tree by merging nodes when all downstream leaves
    yield the same model answer (i.e. the same value for "model_answer" in their "additionals").

    For a leaf node:
      - If it has a non-empty "additionals" list and all entries yield the same "model_answer",
        return that common answer.
      - Otherwise, return None.

    For a non-leaf node:
      - First, compress its children.
      - If every child returns a non-None common answer and all these answers are equal,
        then compress the current node:
          * Clear its "children"
          * Set its "text" attribute to the common answer.
        Return that common answer.
      - Otherwise, leave the node unchanged and return None.

    Parameters:
      tree: dict, an annotated JSON tree (e.g. produced by make_json_tree and annotate_tree).

    Returns:
      A tuple (modified_tree, common_answer) where common_answer is a string if the entire
      subtree yields the same answer, or None otherwise.
    """
    # Base case: leaf node.
    if not tree.get("children"):
        if "additionals" in tree and tree["additionals"]:
            # Extract all model answers from the "additionals" list.
            answers = {entry.get("model_answer") for entry in tree["additionals"]}
            if len(answers) == 1:
                common_answer = answers.pop()
                tree["text"] = common_answer  # update text to the common answer.
                return tree, common_answer
        return tree, None

    # Recursive case: process children first.
    child_answers = []
    for child in tree["children"]:
        # Recursively compress each child.
        _, ans = compress_tree(child)
        child_answers.append(ans)

    # If every child returned a non-None answer and they are all the same...
    if all(ans is not None for ans in child_answers) and len(set(child_answers)) == 1:
        common_answer = child_answers[0]
        # Compress this node by clearing its children and updating its text.
        tree["children"] = []
        tree["text"] = common_answer
        return tree, common_answer
    else:
        # Otherwise, cannot compress this node.
        return tree, None

In [127]:
ctrees=[
    compress_tree(copy.deepcopy(tree))[0]
    for tree in tqdm.tqdm(trees)
]

100%|██████████| 10/10 [00:00<00:00, 119.15it/s]


In [128]:
def get_plot_data(annotated_tree, x_gap=150, y_gap=50):
    """
    Given an annotated JSON tree (with keys including:
      "idlist", "label", "text", "color", "count", "avg_score", "description", "children"),
    compute plotting data.

    This function:
      1. Assigns a unique node id and depth (via BFS), and sets x = depth * x_gap.
      2. Assigns y coordinates via a post‑order pass (leaves get sequential y values; internal nodes get average y).
      3. Flattens the tree into lists of node and edge dictionaries.
         Each node dict contains:
             "id", "x", "y", "label" (using the node’s "text"), "hover_text" (path probability),
             "node_text" (formatted average score), and "color".
         Each edge dict contains:
             "source", "target", "prob", "line_color", "mid_x", "mid_y".
      4. Propagates the root's "description" (instead of "top_text").
    
    Returns:
      A dictionary with keys:
          "description": the root node's description,
          "nodes": list of node dictionaries,
          "edges": list of edge dictionaries.
    """
    from collections import deque

    # --- Step 1: Assign unique ids and depth via BFS ---
    def assign_ids_and_depth(root):
        queue = deque([(root, 0)])
        node_list = []
        next_id = 0
        while queue:
            node, depth = queue.popleft()
            node["depth"] = depth
            node["node_id"] = next_id
            next_id += 1
            node_list.append(node)
            for child in node.get("children", []):
                queue.append((child, depth + 1))
        return node_list

    nodes_list = assign_ids_and_depth(annotated_tree)

    # --- Step 2: Set x coordinate based on depth ---
    for node in nodes_list:
        node["x"] = node["depth"] * x_gap

    # --- Step 3: Assign y coordinates via post‑order traversal ---
    def assign_y(node, current_y):
        if not node.get("children"):
            node["y"] = current_y
            return current_y + y_gap, node["y"]
        else:
            child_ys = []
            for child in node.get("children", []):
                current_y, child_y = assign_y(child, current_y)
                child_ys.append(child_y)
            node["y"] = sum(child_ys) / len(child_ys)
            return current_y, node["y"]

    current_y, _ = assign_y(annotated_tree, 0)

    # --- Step 4: Flatten the tree into nodes and edges ---
    root_count = annotated_tree.get("count", 1)  # Avoid division by zero.
    nodes = []
    edges = []

    def flatten(node):
        hover_text=f"Path probability: {node['count'] / root_count:.2f}" if root_count > 0 else "Path probability: 0.00"
        #if child add model answer
        if len(node.get("children"))==0:
            if "model_answer" in node["additionals"][0]:
                hover_text+=f"<br>Model answer: {node['additionals'][0]['model_answer']}"
        nodes.append({
            "id": node["node_id"],
            "x": node["x"],
            "y": node["y"],
            # Use the precomputed "text" attribute for the full HTML label (parents + current)
            "label": node.get("text", node.get("label", "")),
            "hover_text": hover_text,
            "node_text": f"{node.get('avg_score', 0):.2f}",
            "color": node.get("color", "#000000")
        })
        for child in node.get("children", []):
            # Edge probability is child's count divided by parent's count.
            prob = child["count"] / node["count"] if node["count"] > 0 else 0
            mid_x = (node["x"] + child["x"]) / 2
            mid_y = (node["y"] + child["y"]) / 2
            # Use matplotlib's Reds colormap to compute line color.
            try:
                import matplotlib.colors as mcolors
                import matplotlib.cm as cm
                line_color = mcolors.to_hex(cm.Reds(prob))
            except Exception:
                line_color = "#FF0000"  # Fallback to red.
            edges.append({
                "source": node["node_id"],
                "target": child["node_id"],
                "prob": prob,
                "line_color": line_color,
                "mid_x": mid_x,
                "mid_y": mid_y
            })
            flatten(child)

    flatten(annotated_tree)

    # --- Final: Build the plot data dictionary ---
    description = annotated_tree.get("description", annotated_tree.get("label", ""))
    return {"description": description, "nodes": nodes, "edges": edges}


In [129]:
plot_data = [get_plot_data(tree, x_gap=150, y_gap=50) for tree in trees]

In [130]:
all_traces = []
trace_index_ranges = []
titles = []
n_traces = 0

for dataset in plot_data:
    titles.append(dataset["description"])
    dataset_traces = []
    
    # Create a lookup of nodes by id.
    node_lookup = {node["id"]: node for node in dataset["nodes"]}
    
    # Create edge traces.
    for edge in dataset["edges"]:
        source = edge["source"]
        target = edge["target"]
        x0, y0 = node_lookup[source]["x"], node_lookup[source]["y"]
        x1, y1 = node_lookup[target]["x"], node_lookup[target]["y"]
        
        # Edge line trace.
        line_trace = go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            mode="lines",
            line=dict(width=2, color=edge["line_color"]),
            hoverinfo="none"
        )
        dataset_traces.append(line_trace)
        
        # Midpoint marker for edge hover info.
        midpoint_trace = go.Scatter(
            x=[edge["mid_x"]],
            y=[edge["mid_y"]],
            mode="markers",
            marker=dict(size=30, color="rgba(0,0,0,0)"),
            hoverinfo="text",
            text=[f"Edge Probability: {edge['prob']:.2f}"]
        )
        dataset_traces.append(midpoint_trace)
    
    # Create node trace.
    node_x = []
    node_y = []
    node_customdata = []  # Will hold the node's label for click display.
    node_hover_text = []  # For hover: only the arrival probability.
    node_fill_colors = []
    node_border_colors = []
    node_marker_text = [] # For displaying the average score on the marker.
    
    for node in dataset["nodes"]:
        node_x.append(node["x"])
        node_y.append(node["y"])
        # Use the full HTML text stored in "label" for click display.
        node_customdata.append({"label": node["label"]})
        node_hover_text.append(node["hover_text"])
        # Now simply use the precomputed color.
        node_fill_colors.append(node["color"])
        node_border_colors.append("gray")  # default border color.
        # Use "node_text" to show the average score on the marker.
        node_marker_text.append(node["node_text"])

    node_trace = go.Scatter(
      x=node_x,
      y=node_y,
      mode="markers+text",
      text=node_marker_text,
      textposition="middle center",
      textfont=dict(size=8, color="black"),
      hovertext=node_hover_text,
      hoverinfo="text",
      customdata=node_customdata,
      marker=dict(
          size=24,
          color=node_fill_colors,
          line=dict(width=2, color=node_border_colors)
      )
    )
    dataset_traces.append(node_trace)
    
    # Set initial visibility.
    if len(titles) == 1:
        for t in dataset_traces:
            t["visible"] = True
    else:
        for t in dataset_traces:
            t["visible"] = False
    
    start_idx = len(all_traces)
    all_traces.extend(dataset_traces)
    end_idx = len(all_traces)
    trace_index_ranges.append((start_idx, end_idx))
    n_traces = len(all_traces)

# Build update menu buttons for dataset switching.
buttons = []
for i, (start, end) in enumerate(trace_index_ranges):
    visible = [False] * n_traces
    for j in range(start, end):
        visible[j] = True
    button = dict(
        label="Q. " + str(i),
        method="update",
        args=[{"visible": visible}]
    )
    buttons.append(button)

updatemenus = [dict(
    buttons=buttons,
    direction="down",
    showactive=True,
    x=0.02,
    y=0.98,
    xanchor="left",
    yanchor="top",
    font=dict(size=12),
    pad=dict(r=10, t=10)
)]

fig = go.Figure(
    data=all_traces,
    layout=go.Layout(
        showlegend=False,
        hovermode="closest",
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        updatemenus=updatemenus
    )
)
config = {"displayModeBar": False}

html_div = pyo.plot(fig, include_plotlyjs=True, output_type="div", config=config)

# Prepare JSON strings for HTML.
js_trace_index_ranges = json.dumps(trace_index_ranges)
js_titles = json.dumps(titles)

html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Tree Visualization</title>
  <style>
    html, body {{
      width: 100%;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    #my-plot {{
      position: absolute;
      top: 0;
      left: 0;
      right: 0;
      height: 50vh;
      border: 1px solid #ccc;
      overflow: hidden;
    }}
    #my-lefttext {{
      position: absolute;
      top: 50vh;
      left: 0;
      width: 50%;
      height: 50vh;
      overflow-y: auto;
      background: #eee;
      border: 1px solid #ccc;
      padding: 10px;
      font-family: Arial, sans-serif;
      white-space: pre-wrap;
      box-sizing: border-box;
    }}
    #my-righttext {{
      position: absolute;
      top: 50vh;
      left: 50%;
      width: 50%;
      height: 50vh;
      overflow-y: auto;
      background: #fff;
      border: 1px solid #ccc;
      padding: 10px;
      font-family: Arial, sans-serif;
      white-space: pre-wrap;
      box-sizing: border-box;
    }}
    #bottom-margin {{
      position: absolute;
      top: 75vh;
      left: 0;
      right: 0;
      height: 50vh;
      background: #f5f5f5;
      border: 1px solid #ccc;
      box-sizing: border-box;
    }}
  </style>
</head>
<body>
  <div id="my-plot">
    {html_div}
  </div>
  <div id="my-lefttext">{titles[0]}</div>
  <div id="my-righttext">
    Click on a node to see its label here.<br>
    Hover over nodes to view arrival probability.<br>
    Hover over edges for probability info.
  </div>
  <script>
    var plotDiv = document.getElementsByClassName("plotly-graph-div")[0];
    var leftBox = document.getElementById("my-lefttext");
    var rightBox = document.getElementById("my-righttext");

    var traceIndexRanges = {js_trace_index_ranges};
    var datasetTitles = {js_titles};

    // Save default border colors for node traces.
    var defaultBorderColors = {{}};
    plotDiv.data.forEach(function(trace, idx) {{
      if(trace.marker && trace.marker.line && Array.isArray(trace.marker.line.color)) {{
        defaultBorderColors[idx] = trace.marker.line.color.slice();
      }}
    }});

    // Function to highlight a node by changing its border color to pink.
    function highlightNode(traceIndex, pointIndex) {{
      var nodeTrace = plotDiv.data[traceIndex];
      if(!nodeTrace || !nodeTrace.marker || !nodeTrace.marker.line) return;
      var newBorderColors = defaultBorderColors[traceIndex].slice();
      newBorderColors[pointIndex] = "pink";
      Plotly.restyle(plotDiv, {{ "marker.line.color": [newBorderColors] }}, [traceIndex]);
    }}

    // On click, display the node's label and update border highlighting.
    plotDiv.on('plotly_click', function(data) {{
      if(data.points.length > 0) {{
        var pt = data.points[0];
        var label = pt.customdata.label;
        if (!label) return;
        rightBox.innerHTML = label;
        highlightNode(pt.curveNumber, pt.pointIndex);
      }}
    }});

    // Update left box with current dataset title after re-render.
    plotDiv.on('plotly_afterplot', function() {{
      setTimeout(function() {{
        var currentDataset = null;
        for(var i = 0; i < traceIndexRanges.length; i++) {{
          var range = traceIndexRanges[i];
          var allVisible = true;
          for(var j = range[0]; j < range[1]; j++) {{
            if(!plotDiv.data[j].visible) {{
              allVisible = false;
              break;
            }}
          }}
          if(allVisible) {{
            currentDataset = i;
            break;
          }}
        }}
        if(currentDataset !== null) {{
          leftBox.innerHTML = datasetTitles[currentDataset];
        }}
      }}, 80);
    }});
  </script>
</body>
</html>
"""

with open(save_path, "w", encoding="utf-8") as f:
    f.write(html_template)